In [23]:
# load a coffea file
from coffea.util import load
import os
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib
import re
matplotlib.rcParams["figure.dpi"] = 300


# Load file

In [24]:
# input = "/work/rcereghetti/out_vbfhh4b/output_vbf_matching5/output_all.coffea"
# o = load(f"{input}")
input = "/work/rcereghetti/out_vbfhh4b/outputDNNsampleSemiTight/output_all.coffea"
# input = "/work/rcereghetti/out_vbfhh4b/out_test9/output_all.coffea"
# input = "/work/mmalucch/out_hh4b/out_vbf_ggf_dnn_full/output_all.coffea"
o = load(f"{input}")

# Function to extract the cut string

In [25]:
def remove_if_found(word, substring):
    if substring in word:
        return word.replace(substring, '')
    return word  # Restituisce la parola invariata se la sottostringa non è trovata

# Create a dictionary with number of events for every step and region

In [26]:
cuts_dict = {}

pattern = r'^[^_]+_[^_]+_([^_]+)_region'

for k in o["cutflow"].keys():    
    print(k)
    if "VBFtight" in k and "VBFtight_region" not in k:
        a = remove_if_found(k, "4b_")
        b = remove_if_found(a, "_region")
        c = remove_if_found(b, "VBFtight_")
        string = c
    else: string = k
    for kk in o["cutflow"][k].keys():
        print("a", kk)
        if isinstance(o['cutflow'][k][kk], int):
            cuts_dict[string] = o['cutflow'][k][kk]
        elif "VBFHHto4B" in kk:
            cuts_dict[string] = o['cutflow'][k][kk]['VBF_HHto4B']


initial
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE
skim
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE
presel
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE
4b_region
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE
4b_semiTight_LeadingPt_region
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE
4b_semiTight_LeadingMjj_region
a GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE
a VBFHHto4B_CV_1_C2V_1_C3_1_2022_postEE


# Print the efficiency 

In [27]:
a = o["cutflow"]["initial"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE"]
b = o["cutflow"]["4b_region"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE"]["GluGlutoHHto4B"]
print(b/a)

0.06995230524642289


In [28]:
cuts_list = list(cuts_dict)
print(cuts_dict)

for i in range(3, len(cuts_list)):
    print(f"Efficiency for {cuts_list[i]} with respect to 4b_region: {cuts_dict[cuts_list[i]] / cuts_dict['4b_region']}")

{'initial': 204633, 'skim': 44944, 'presel': 24927, '4b_region': 7455, '4b_semiTight_LeadingPt_region': 3912, '4b_semiTight_LeadingMjj_region': 4168}
Efficiency for 4b_region with respect to 4b_region: 1.0
Efficiency for 4b_semiTight_LeadingPt_region with respect to 4b_region: 0.524748490945674
Efficiency for 4b_semiTight_LeadingMjj_region with respect to 4b_region: 0.5590878604963112


# Check signal efficiency 3rd and 4th jet

In [29]:
print(list(o["columns"].keys()))
sample=list(o["columns"].keys())[0]
dataset=list(o["columns"][sample].keys())[0]
col={}
for i in range(len(o["columns"][sample][dataset].keys())):
    category = list(o["columns"][sample][dataset].keys())[i]
    col[list(o["columns"][sample][dataset].keys())[i]] = o["columns"][sample][dataset][category]

print(o["columns"]["GluGlutoHHto4B"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE"].keys())

['GluGlutoHHto4B', 'VBF_HHto4B']
dict_keys(['4b_region', '4b_semiTight_LeadingPt_region', '4b_semiTight_LeadingMjj_region'])


In [32]:
colMjj = o["columns"]["GluGlutoHHto4B"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE"]["4b_semiTight_LeadingMjj_region"]
colPt = o["columns"]["GluGlutoHHto4B"]["GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00__2022_postEE"]["4b_semiTight_LeadingPt_region"]

jj_mass = colMjj["events_JetVBFLeadingMjjNotFromHiggs_jjMass"].value
print(len(jj_mass))


5


In [9]:
# index = col["4b_region"]["JetVBF_matched_index"].value
# tot = np.zeros(11)
# # print(tot)
# for i in range(len(index)):
#     tot[int(index[i])]+=1
# # for i in range(len(tot)):
# #     print(f"Index {i}: {tot[i]/len(index)}")
# print(np.sum(tot[6:]),len(index), np.sum(tot[6:])/len(index))
# plt.hist(index, 11)
# plt.grid()
# plt.show()

In [10]:
# regions = []
# for k in o["cutflow"].keys():
#     regions.append(k)

# for i in range(3, len(regions)):
#     index = col[regions[i]]["JetVBF_matched_index"].value
#     tot = np.zeros(11)
#     for j in range(len(index)):
#         tot[int(index[j])]+=1
#     print(f"{cuts_list[i]}: {np.sum(tot[6:])} {len(index)} {np.sum(tot[6:])/len(index)}")


In [ ]:
index = col["4b_region"].keys()

print(index)

dict_keys(['weight', 'events_HH', 'events_JetVBFLeadingPtNotFromHiggs_deltaEta', 'events_JetVBFLeadingMjjNotFromHiggs_deltaEta', 'events_JetVBFLeadingPtNotFromHiggs_jjMass', 'events_JetVBFLeadingMjjNotFromHiggs_jjMass', 'events_HH_deltaR', 'events_H1j1_deltaR', 'events_H1j2_deltaR', 'events_H2j1_deltaR', 'events_H2j2_deltaR', 'events_HH_centrality', 'HiggsLeading_pt', 'HiggsLeading_eta', 'HiggsLeading_phi', 'HiggsLeading_mass', 'HiggsSubLeading_pt', 'HiggsSubLeading_eta', 'HiggsSubLeading_phi', 'HiggsSubLeading_mass', 'Jet_N', 'Jet_index', 'Jet_pt', 'Jet_btagPNetQvG', 'Jet_eta', 'Jet_btagPNetB', 'Jet_phi', 'Jet_mass', 'JetGoodFromHiggsOrdered_N', 'JetGoodFromHiggsOrdered_index', 'JetGoodFromHiggsOrdered_pt', 'JetGoodFromHiggsOrdered_btagPNetQvG', 'JetGoodFromHiggsOrdered_eta', 'JetGoodFromHiggsOrdered_btagPNetB', 'JetGoodFromHiggsOrdered_phi', 'JetGoodFromHiggsOrdered_mass', 'JetVBFLeadingPtNotFromHiggs_N', 'JetVBFLeadingPtNotFromHiggs_index', 'JetVBFLeadingPtNotFromHiggs_pt', 'JetVBFL

In [ ]:
eta = col["4b_region"]["JetVBFLeadingMjjNotFromHiggs_eta"].value
deltaEta = col["4b_region"]["events_JetVBFLeadingPtNotFromHiggs_deltaEta"].value
events = col["4b_region"]["Jet_N"].value
sum = 0
print(events)
print(len(deltaEta)/events)
for i in range(0, int(len(eta)/2)):
    if (abs(eta[2*i]) < 4.7)& (abs(eta[2*i+1]) < 4.7):
    # if (abs(eta[2*i]) > 3.5)  & (abs(eta[2*i]) < 4.7) & (abs(eta[2*i+1]) > 3.5)  & (abs(eta[2*i+1]) < 4.7):
        sum +=1
        # print("eta0", eta[2*i], "eta1", eta[2*i+1])
print(sum/(len(eta)/2))
               

[ 7  7  6 12  6  6  6  6  6  7  8  6  9 10  7  7  7  6  9 10  8  7  8  9
  7  9  7  7 10  7  6  9  7  6  8  7  7  7  8  6 10  7  9  6  7  7  6  7
  6  9  6  7  7  6  8  6  7  6  8  8  9  8  6  9  7 12  6  9  6  6 10  7
  6  8  7  7  7  8  7 10  7  8  7  7  6  8  8  8]
[12.57142857 12.57142857 14.66666667  7.33333333 14.66666667 14.66666667
 14.66666667 14.66666667 14.66666667 12.57142857 11.         14.66666667
  9.77777778  8.8        12.57142857 12.57142857 12.57142857 14.66666667
  9.77777778  8.8        11.         12.57142857 11.          9.77777778
 12.57142857  9.77777778 12.57142857 12.57142857  8.8        12.57142857
 14.66666667  9.77777778 12.57142857 14.66666667 11.         12.57142857
 12.57142857 12.57142857 11.         14.66666667  8.8        12.57142857
  9.77777778 14.66666667 12.57142857 12.57142857 14.66666667 12.57142857
 14.66666667  9.77777778 14.66666667 12.57142857 12.57142857 14.66666667
 11.         14.66666667 12.57142857 14.66666667 11.         11.
  9.77777

In [ ]:



# HH_centrality = col["4b_region"]["events_HH_centrality"].value
# jj_eta = col["4b_region"]["JetVBFLeadingPtNotFromHiggs_eta"].value
# H1_eta= col["4b_region"]["HiggsLeading_eta"].value
# H2_eta= col["4b_region"]["HiggsSubLeading_eta"].value
# deltaR = col["4b_region"]["events_H1j2_deltaR"].value
# for i in range(0, 3):
#     print(jj_eta[2*i], jj_eta[2*i+1])
#     eta_avg = (jj_eta[2*i] + jj_eta[2*i+1]) / 2
#     delta_eta
#     a = H1_eta - eta_avg
#     b = H2_eta - eta_avg

#     print(HH_centrality[i])

#     print("\n")
for i in range(len(HH_centrality)):
    if HH_centrality[i] < 0.1:
        print((HH_centrality[i]))
print(len(HH_centrality))

# np.exp(
#                     -(((self.events.HiggsLeading.eta - JetVBFLeadingPtNotFromHiggs_etaAverage)**2 
#                         -(self.events.HiggsSubLeading.eta - JetVBFLeadingPtNotFromHiggs_etaAverage)**2
#                     ) / (self.events.JetVBFLeadingPtNotFromHiggs_deltaEta)**2

# sum = 0
# for i in range(len(index)):
#     if index[i] == -999: sum+=1
# print(sum)
# tot = np.zeros(12)
# for i in range(len(index)):
#     if index[i] >= 0:
#         tot[int(index[i])]+=1
# for i in range(len(tot)):
#     print(f"Index {i}: {tot[i]}") #/len(index))
# print(tot)
# print(np.sum(tot[6:]),len(index), np.sum(tot[6:])/len(index))

# index_array = np.array(index)
# index_not_none = index_array[index_array >= 0]

# plt.hist(index_not_none, 11)
# plt.grid()
# plt.show()
# print(len(index))
# print(len(index_not_none))

NameError: name 'HH_centrality' is not defined

In [60]:
jetBtag = col["4b_region"]["JetVBF_matched_btagPNetB"].value
nJetVBF = col["4b_region"]["events_nJetVBF_matched"].value
jetBtag_unflatten = ak.unflatten(jetBtag, nJetVBF)
print(ak.sum(nJetVBF) / (2 * len(nJetVBF)))
print(jetBtag_unflatten)
mask_two = nJetVBF == 2
print(ak.sum(mask_two) / len(nJetVBF))
mask_one = nJetVBF == 1
print(ak.sum(mask_one) / len(nJetVBF))
mask_zero = nJetVBF == 0
print(ak.sum(mask_zero) / len(nJetVBF))
weight = col["4b_region"]["weight"].value
mask_negative = weight < 0
print(weight[mask_negative])
mask_not_none = jetBtag > -2
mask_medium = jetBtag >0.2605
# print(len(jetBtag[mask_medium]) /len(jetBtag[mask_not_none]))
# plt.hist(jetBtag)

KeyError: 'JetVBF_matched_btagPNetB'

In [ ]:
FromHiggs_pt = col["4b_VBFtight_njet_vbf_region"]["JetGoodFromHiggsOrdered_pt"].value
# print(FromHiggs_pt)
VBF_pt = col["4b_VBFtight_njet_vbf_region"]["JetVBFLeadingPtNotFromHiggs_pt"].value
print(VBF_pt)

[170.54393   21.140991 130.32283  ... 160.3455   346.12814   38.199215]


In [ ]:
VBFnotLeadingPtFromHiggs_index = col["4b_VBFtight_njet_vbf_region"]["JetVBFLeadingPtNotFromHiggs_index"].value
VBFnotFromHiggs_index = col["4b_VBFtight_njet_vbf_region"]["JetVBFNotFromHiggs_index"].value
stop = 8
FromHiggs_index = col["4b_VBFtight_njet_vbf_region"]["JetGoodFromHiggsOrdered_index"].value
VFBgeneralSelection_index = col["4b_VBFtight_njet_vbf_region"]["JetVBF_generalSelection_matched_index"].value
VFBgeneralSelection_index2 = col["4b_region"]["JetVBF_generalSelection_matched_index"].value

# print(np.reshape(VFBgeneralSelection_index, (-1, 2))[0:stop])
# print(np.reshape(VBFnotLeadingPtFromHiggs_index, (-1, 2))[0:stop])
# # print(np.reshape(VBF_index[0:stop], (-1,2)))
print(FromHiggs_index[0:stop])
print(VBFnotFromHiggs_index[0:stop + 2])
print(VBFnotLeadingPtFromHiggs_index[0:stop])
print(VFBgeneralSelection_index[0:stop])
print(VFBgeneralSelection_index2[0:stop])
print()

[[2 4 1 3]
 [1 3 6 7]
 [1 3 2 6]
 [0 4 2 5]
 [2 1 3 5]
 [0 3 2 4]
 [0 4 1 5]
 [0 5 1 4]]
[0 5 6 0 2 5 4 8 0 4]
[0 5 0 2 0 4 1 3]
[0. 6. 0. 2. 4. 5. 1. 3.]
[1. 0. 6. 0. 2. 4. 3. 0.]



# VBF matching efficiency

In [ ]:
VBFnotLeadingPtFromHiggs_index = col["4b_region"]["JetVBFLeadingPtNotFromHiggs_index"].value
VFBgeneralSelection_index = col["4b_region"]["JetVBF_generalSelection_matched_index"].value
VBF_matched_index = col["4b_region"]["JetVBF_matched_index"].value
Jet_index = col["4b_region"]["Jet_index"].value
Jet_N = col["4b_region"]["Jet_N"].value

print("Matching efficinecy in 4b_region:", "\n")
print("VBF_matched:", len(VBF_matched_index) / (2 * cuts_dict["4b_region"]))
print("VBF_generalSelection_matched:", len(VFBgeneralSelection_index) / (2*len(Jet_N)))
print("VBFLeadingPtnotFromHiggs:", len(VBFnotLeadingPtFromHiggs_index) / (2*len(Jet_N)))
print(len(Jet_N))
print(cuts_dict["4b_region"])

Matching efficinecy in 4b_region: 

VBF_matched: 0.8606975184439973
VBF_generalSelection_matched: 0.8511066398390342
VBFLeadingPtnotFromHiggs: 1.0
7455
7455


In [ ]:
VFBgeneralSelection_index = col["4b_VBFtight_njet_vbf_region"]["JetVBF_generalSelection_matched_index"].value
VBF_matched_index = col["4b_VBFtight_njet_vbf_region"]["JetVBF_matched_index"].value
Jet_index = col["4b_VBFtight_njet_vbf_region"]["Jet_index"].value

print("Matching efficinecy in 4b_VBFtight_njet_vbf_region:", "\n")
print("VBF_matched:", len(VBF_matched_index) / len(Jet_index))
print("VBF_generalSelection_matched:", len(VFBgeneralSelection_index) / len(Jet_index))
print("VBFLeadingPtnotFromHiggs:", len(VBFnotLeadingPtFromHiggs_index) / len(Jet_index))

Matching efficinecy in 4b_VBFtight_njet_vbf_region: 

VBF_matched: 0.27022904519398727
VBF_generalSelection_matched: 0.2669815730558221
VBFLeadingPtnotFromHiggs: 0.27022904519398727


In [ ]:
VBFnotFromHiggs_eta = col["4b_region"]["events_JetVBFLeadingPtNotFromHiggs_deltaEta"].value
print(VBFnotFromHiggs_eta)
# print(len(VBFnotFromHiggs_eta[0::2]))
# print(len(VBFnotFromHiggs_eta[1::2]))
# print(abs(VBFnotFromHiggs_eta[0::2]-VBFnotFromHiggs_eta[1::2]))

[4.7905273 4.6987305 3.9665527 ... 2.9509277 1.0402164 7.2128906]


In [ ]:
VBFnotFromHiggs_eta = col["4b_region"]["even"].value